In [1]:
import urllib2
import json
import numpy as np
import pylab
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Spectral11
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def pullDates():
    api_key='2B175EF8A024C543D84E2A581F139C0A'        
    url='http://api.eia.gov/series/?api_key='+ api_key+'&series_id=ELEC.PRICE.CA-ALL.M'
    json_obj= urllib2.urlopen(url)
    data = json.load(json_obj)
    dates=[]     
    for i in data['series'][0]['data']:
        dates.append(i[0])
    #statePrice=pd.DataFrame(np.nan)   
    return dates
dates=pullDates()
print dates

[u'201603', u'201602', u'201601', u'201512', u'201511', u'201510', u'201509', u'201508', u'201507', u'201506', u'201505', u'201504', u'201503', u'201502', u'201501', u'201412', u'201411', u'201410', u'201409', u'201408', u'201407', u'201406', u'201405', u'201404', u'201403', u'201402', u'201401', u'201312', u'201311', u'201310', u'201309', u'201308', u'201307', u'201306', u'201305', u'201304', u'201303', u'201302', u'201301', u'201212', u'201211', u'201210', u'201209', u'201208', u'201207', u'201206', u'201205', u'201204', u'201203', u'201202', u'201201', u'201112', u'201111', u'201110', u'201109', u'201108', u'201107', u'201106', u'201105', u'201104', u'201103', u'201102', u'201101', u'201012', u'201011', u'201010', u'201009', u'201008', u'201007', u'201006', u'201005', u'201004', u'201003', u'201002', u'201001', u'200912', u'200911', u'200910', u'200909', u'200908', u'200907', u'200906', u'200905', u'200904', u'200903', u'200902', u'200901', u'200812', u'200811', u'200810', u'200809'

In [4]:
def pullDataByState(state):
    api_key='2B175EF8A024C543D84E2A581F139C0A'        
    url='http://api.eia.gov/series/?api_key='+ api_key+'&series_id=ELEC.PRICE.'+str(state)+ '-ALL.M'
    json_obj= urllib2.urlopen(url)
    data = json.load(json_obj)
    prices=[]     
    for i in data['series'][0]['data']:
        prices.append(i[1])
    #statePrice=pd.DataFrame(np.nan)   
    return prices
#print pullDataByState('CA')

In [5]:
States=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

def PriceDF():
    df=pd.DataFrame(np.nan, index=pullDates(), columns=States)    
    for state in States:
        df[state]=pullDataByState(state)
    return df
    
a=PriceDF()
#print a

In [6]:
stats=a.describe()

In [13]:
mean_price=stats.ix['mean']
    
#print mean_price.order(ascending=False)
#print mean_price['HI']
#print mean_price.describe()

Index([u'AL', u'AK', u'AZ', u'AR', u'CA', u'CO', u'CT', u'DE', u'FL', u'GA',
       u'HI', u'ID', u'IL', u'IN', u'IA', u'KS', u'KY', u'LA', u'ME', u'MD',
       u'MA', u'MI', u'MN', u'MS', u'MO', u'MT', u'NE', u'NV', u'NH', u'NJ',
       u'NM', u'NY', u'NC', u'ND', u'OH', u'OK', u'OR', u'PA', u'RI', u'SC',
       u'SD', u'TN', u'TX', u'UT', u'VT', u'VA', u'WA', u'WV', u'WI', u'WY'],
      dtype='object')

In [24]:
svg = open('usaLow.svg', 'r').read()
soup = BeautifulSoup(svg, 'html.parser')# check that 'html.parser' will work next tim -  , selfClosingTags=['defs','sodipodi:namedview'])
#print soup.prettify()
paths = soup.findAll('path')
colors=['#f1eef6', '#d7b5d8', '#df65b0', '#dd1c77', '#980043']
path_style = 'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;stroke-width:0.1;stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;marker-start:none;stroke-linejoin:bevel;fill:'
#print soup.prettify()
#print paths[0]

In [25]:
for p in paths:
    try:
        rate = mean_price[p['id'][3:]]
    except KeyError:
        continue         
    
    if rate > 15:
        color_class = 4
    elif rate > 12:
        color_class = 3
    elif rate > 9:
        color_class = 2
    elif rate > 6:
        color_class = 1
    else:
        color_class = 0
    color = colors[color_class]
    p['style'] = path_style + color
    #p['fill'] = color

In [26]:
fp = open('mean_price.svg','wb')
fp.write(str(soup))
fp.close()